# 真正的 Few-Shot Learning 训练与评估

## 关键区别

| | 之前（错误） | 现在（正确） |
|---|---|---|
| 训练 | 200 类 | 100 类 (base) |
| 测试 | 同样 200 类的不同图片 | 另外 100 类 (novel) |
| 测的是 | 泛化到新图片 | **泛化到新类别** |

这才是真正的 few-shot 评估！

In [ ]:
# 1. 检查 GPU
!nvidia-smi

In [ ]:
# 2. 克隆代码
import os
if not os.path.exists('PartGraph-FewShot'):
    !git clone https://github.com/alltobebetter/PartGraph-FewShot.git
else:
    !cd PartGraph-FewShot && git pull
%cd PartGraph-FewShot

In [ ]:
# 3. 安装依赖
!pip install -q -r requirements.txt

In [ ]:
# 4. 下载 CUB-200 数据集
import os
os.makedirs('data', exist_ok=True)
%cd data
if not os.path.exists('CUB_200_2011'):
    !wget -q https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz
    !tar -xzf CUB_200_2011.tgz
    print('Dataset downloaded!')
else:
    print('Dataset exists.')
%cd ..

---
## 训练 (只在 Base Classes 上)

In [ ]:
# 5. 训练 Baseline (无 GNN)
!python src/train_fewshot.py \
    --data_root ./data \
    --output_dir ./checkpoints_baseline_fewshot \
    --batch_size 16 \
    --epochs 50 \
    --num_slots 8 \
    --slot_dim 256 \
    --slot_iters 3 \
    --lr 1e-4

In [ ]:
# 6. 训练 GNN-in-the-Loop
!python src/train_fewshot.py \
    --data_root ./data \
    --output_dir ./checkpoints_gnn_fewshot \
    --use_gnn \
    --batch_size 16 \
    --epochs 50 \
    --num_slots 8 \
    --slot_dim 256 \
    --slot_iters 3 \
    --gnn_start_iter 1 \
    --lr 5e-5

---
## 评估 (在 Novel Classes 上 - 从未见过的类别!)

In [ ]:
# 7. 评估 Baseline
!python src/eval_fewshot_novel.py \
    --data_root ./data \
    --checkpoint ./checkpoints_baseline_fewshot/best_model.pth \
    --num_slots 8 \
    --slot_dim 256 \
    --n_way 5 \
    --k_shot 5 \
    --n_episodes 600

In [ ]:
# 8. 评估 GNN-in-the-Loop
!python src/eval_fewshot_novel.py \
    --data_root ./data \
    --checkpoint ./checkpoints_gnn_fewshot/best_model.pth \
    --use_gnn \
    --num_slots 8 \
    --slot_dim 256 \
    --n_way 5 \
    --k_shot 5 \
    --n_episodes 600

In [ ]:
# 9. 1-shot 评估 (更难的设置)
print("=== 5-way 1-shot (harder) ===")
!python src/eval_fewshot_novel.py \
    --data_root ./data \
    --checkpoint ./checkpoints_gnn_fewshot/best_model.pth \
    --use_gnn \
    --num_slots 8 \
    --slot_dim 256 \
    --n_way 5 \
    --k_shot 1 \
    --n_episodes 600

In [ ]:
# 10. 查看可视化
from IPython.display import Image, display
import glob

for folder in ['checkpoints_baseline_fewshot', 'checkpoints_gnn_fewshot']:
    print(f'\n=== {folder} ===')
    vis_files = sorted(glob.glob(f'./{folder}/epoch_*_vis.png'))
    if vis_files:
        display(Image(filename=vis_files[-1]))

---
## 预期结果

在 **Novel Classes** 上的 5-way 5-shot 准确率：

| 方法 | 准确率 |
|------|--------|
| Random | 20% |
| Backbone only | ~50-55% |
| Slot Attention | ~55-65% |
| GNN-in-Loop | ~60-70%? |

注意：这比之前的 88% 低很多，因为现在是**真正的 few-shot**——测试类别从未在训练中见过！